### Update Building Scene Layers for Construction Monitoring

#### This Notebook helps to update existing building scene layer using new revit files to update construction progress of statino structures

#### -------------- Steps ----------------- ####
#### A. Process new revit files (with new construction status)
1. Define Projection to PRS92
2. BIM to Geodatabase (file geodatabase)
3. Make Building Layers

#### B. Update Existing Building Layers
##### For Each building component layer, repeat the follow:
1. Join Field using new layer ([transfer field = 'Status'])
2. Update 'Status' for selective rows
3. Delete 'Status1'

#### C. Update Existing Building Layers
##### Truncate existing building layers in SDE and append the copied
1. Truncate existing building layers in enterprise geodatabase
2. Append the copied to the truncated building layers

#### E. Delete temporary file geodatabase
1. Delete file geodatabase created during the process 'A'
2. Delete file geodatabase copied during the process 'C'

In [1]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
from pathlib import Path
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984 Web Mercator (auxiliary sphere)")
arcpy.env.geographicTransformations = "PRS_1992_To_WGS_1984_1"

home = Path.home()
directory = r'OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2'
workspace = os.path.join(home, directory)

BuildingLayerName = "temporary_BIM_layers_new"

### A. Process new revit files

In [2]:
out_gdb_path = os.path.join(workspace,"During-Construction_nscrexn2.gdb")
out_dataset_name = "temporary_BIM_dataset"

In [3]:
# 2. BIM File to Geodatabase
## This saves all Revit models to enterprise geodatabase
RevitWD = os.path.join(home, "Dropbox/01-Railway/02-NSCR-Ex/01-N2/01-Basedata/09-Station_Structure/99-BIM Model") 
Revit1 = os.path.join(RevitWD,"N-01/NSCR-GCR-N01-APLSTN-MOD-ST-000001.rvt")
#Revit2 = os.path.join(RevitWD, "N-01/NSCR-GCR-N01-CLPSTN-MOD-ST-000001.rvt")

#inputRevit = [Revit1,Revit2]
inputRevit = [Revit1]

# Choose enterprise geodatabase for Output Geodatabase
spatial_reference = "PRS_1992_Philippines_Zone_III"

arcpy.BIMFileToGeodatabase_conversion(inputRevit, out_gdb_path, out_dataset_name, spatial_reference)


<Result 'C:\\Users\\eiji.LAPTOP-KSD9P6CP\\OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2\\During-Construction_nscrexn2.gdb\\temporary_BIM_dataset'>

In [4]:
# 3. Make Building Layer
out_dataset = os.path.join(out_gdb_path, out_dataset_name)
arcpy.MakeBuildingLayer_management(out_dataset,BuildingLayerName)

<Result 'temporary_BIM_layers_new'>

In [ ]:
# 4. 'remove empty layer' (Is this necessary?)

In [5]:
# 5. Specificy input Tables
buildingLayerNameArchitectural = BuildingLayerName + "\\" + "Architectural"
buildingLayerNameStructural = BuildingLayerName + "\\" + "Structural"

input_features = [buildingLayerNameStructural + "\\" + "StructuralColumns",
               buildingLayerNameStructural + "\\" + "StructuralFoundation",
               buildingLayerNameStructural + "\\" + "StructuralFraming",
               buildingLayerNameArchitectural + "\\" + "Columns",
               buildingLayerNameArchitectural + "\\" + "Floors",
               buildingLayerNameArchitectural + "\\" + "Walls"
              ]

basename_input = []
for feature in input_features:
    basename = os.path.basename(feature)
    basename_input.append(basename)
    
basename_input

['StructuralColumns', 'StructuralFoundation', 'StructuralFraming', 'Columns', 'Floors', 'Walls']

### C. Update Existing Building Layers in SDE

In [6]:
# 1. Specify copied Building Layers (to be updated)
existingBuildingLayer = "N2_Station_Structure_practice"
existingBuildingLayerNameArchitectural = existingBuildingLayer + "\\" + "Architectural"
existingBuildingLayerNameStructural = existingBuildingLayer + "\\" + "Structural"

target_features = [existingBuildingLayerNameStructural + "\\" + "StructuralColumns",
               existingBuildingLayerNameStructural + "\\" + "StructuralFoundation",
               existingBuildingLayerNameStructural + "\\" + "StructuralFraming",
               existingBuildingLayerNameArchitectural + "\\" + "Columns",
               existingBuildingLayerNameArchitectural + "\\" + "Floors",
               existingBuildingLayerNameArchitectural + "\\" + "Walls"
              ]

target_features

['N2_Station_Structure_practice\\Structural\\StructuralColumns', 'N2_Station_Structure_practice\\Structural\\StructuralFoundation', 'N2_Station_Structure_practice\\Structural\\StructuralFraming', 'N2_Station_Structure_practice\\Architectural\\Columns', 'N2_Station_Structure_practice\\Architectural\\Floors', 'N2_Station_Structure_practice\\Architectural\\Walls']

In [7]:
#** IMPORTANT
## If this process failed for specific layer, it implies that you need to unlock the layer's connection
## in the enterprise geodabase. Only database administrator is allowed to do this.

# 2. Join Field
## Join updating new building layers to existing layers
## When starting to receive real revit models, make sure to use 'Status'

deleteTransferField = 'Family'
#deleteTransferField = 'Status'
joinField = 'ObjectId'

for feature in target_features:
    #arcpy.DeleteField_management(feature, [deleteTransferField])
    
    # Get input_feature corresponding to target_feature
    basename = os.path.basename(feature)
    index = basename_input.index(basename)
    joinTable = input_features[index]
    
    # Join
    arcpy.management.JoinField(feature, joinField, joinTable, joinField, deleteTransferField)

In [8]:
# 3. Update 'Status' using 'Status1'
## This process is needed because only subset of rows may be updated.
## In this case, we should check if 'Status' must be updated using new information.
## Otherwise, keep existing 'Status'

tempField = 'Family_1'
# tempField = 'Status_1'

for feature in target_features:
    with arcpy.da.UpdateCursor(feature, [deleteTransferField, tempField]) as cursor:
        for row in cursor:
            if row[1] is None:
                row[0] = row[0]
            elif row[0] is not row[1]:
                row[0] = row[1]
            cursor.updateRow(row)

In [9]:
# delete tempField 
for feature in target_features:
    arcpy.DeleteField_management(feature, tempField)

### C. Delete temporary feature dataset

In [ ]:
# You should manually delete temporary feature dataset in your local file geodatabase